# Graphiti Agent

그래피티(Graphiti)는 AI 에이전트를 위해 설계된 시간 인식 지식 그래프 구축을 위한 파이썬 프레임워크입니다. 일괄 재계산 없이 지식 그래프에 대한 실시간 증분 업데이트를 가능하게 하여, 관계와 정보가 시간에 따라 진화하는 동적 환경에 적합합니다.

![](https://steemitimages.com/300x0/https://github.com/gongwon-nayeon/graphiti-agent-tutorial/raw/main/graphiti_agent.png)

참고: https://github.com/gongwon-nayeon/graphiti-agent-tutorial/blob/main/graphiti_agent.ipynb

이번 튜토리얼에서는 [Neo4j Sandbox](https://sandbox.neo4j.com/)를 사용합니다.
> Neo4j Sandbox란? **온라인 그래프 데이터베이스**로, 무료 클라우드 기반 Neo4j 인스턴스를 구축할 수 있기 떄문에 간편하게 리소스를 빌려 그래프DB를 실험 할 수 있습니다.

## 설치하기

필요한 패키지((graphiti, langgraph, langchain, 등)을 설치합니다.

In [1]:
%%capture --no-stderr
%pip install -qU langgraph langchain-core langchain-openai 
%pip install -qU graphiti-core

## 환경 설정

graphiti를 실행하는데 필요한 환경변수를 설정합니다. graphiti는 OpenAI를 사용하므로 OpenAI API Key가 필요합니다.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"

neo4j_uri = os.environ.get("NEO4J_URI")
neo4j_user = os.environ.get("NEO4J_USERNAME")
neo4j_password = os.environ.get("NEO4J_PASSWORD")

if not neo4j_uri or not neo4j_user or not neo4j_password:
    raise ValueError("NEO4J_URI, NEO4J_USERNAME, and NEO4J_PASSWORD must be set")

In [3]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

## Graphiti 사용하기

In [4]:
from graphiti_core import Graphiti

graphiti = Graphiti(neo4j_uri, neo4j_user, neo4j_password)

await graphiti.build_indices_and_constraints()

2025-10-20 20:54:45 - neo4j.notifications - INFO - Received notification from DBMS server: <GqlStatusObject gql_status='00NA0', status_description="note: successful completion - index or constraint already exists. The command 'CREATE RANGE INDEX valid_at_episodic_index IF NOT EXISTS FOR (e:Episodic) ON (e.valid_at)' has no effect. The index or constraint specified by 'RANGE INDEX valid_at_episodic_index FOR (e:Episodic) ON (e.valid_at)' already exists.", position=None, raw_classification='SCHEMA', classification=<NotificationClassification.SCHEMA: 'SCHEMA'>, raw_severity='INFORMATION', severity=<NotificationSeverity.INFORMATION: 'INFORMATION'>, diagnostic_record={'_classification': 'SCHEMA', '_severity': 'INFORMATION', 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: 'CREATE INDEX valid_at_episodic_index IF NOT EXISTS FOR (n:Episodic) ON (n.valid_at)'
2025-10-20 20:54:45 - neo4j.notifications - INFO - Received notification from DBMS server: <GqlStatusObject gq

## 1. 에피소드 추가

에피소드는 Graphiti의 기본 단위입니다.

**에피소드 유형**은 다음과 같습니다.
- `text`: 비정형 택스트 데이터
- `message`: 대화형 메시지 형식
- `json`: 구조화된 데이터

In [5]:
from graphiti_core.nodes import EpisodeType

episodes = [
    {
        "type": EpisodeType.json,
        "name": "OpenAI",
        "content": {
            "name": "OpenAI",
            "founded": "2015-12-8",
            "founders": "Sam Altman",
            "type": "AI 연구 개발 기업",
            "official_website": "https://openai.com/",
        },
        "description": "article metadata",
    },
    {
        "type": EpisodeType.text,
        "name": "OpenAI의 기본 모델",
        "content": "2024년 5월 기준, OpenAI의 기본 모델은 GPT-4o입니다.",
        "description": "news article",
    },
]

In [ ]:
# 에피소스 등록하기
import json
from datetime import datetime, timezone


for i, episode in enumerate(episodes):
    await graphiti.add_episode(
        name=episode["name"],
        episode_body=episode["content"]
        if isinstance(episode["content"], str)
        else json.dumps(episode["content"]),
        source=episode["type"],
        source_description=episode["description"],
        reference_time=datetime.now(timezone.utc),
    )
    print(f"Added episode: {episode['name']} ({episode['type'].value})")